# Freesurfer space to native space using mri_vol2vol

`05-freesurfer-to-native-space.ipynb` ver. 2020-03-11

In [1]:
%matplotlib inline
import os
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
from os.path import expanduser, join, basename, split
home = expanduser('~') # To make path to local home directory
import sys
sys.path.append('.')   # path to utils.py
import utils
import time
import shutil  # copy files

### We will use the Freesurfer 6.0 tree previously run for the Toro project

In [2]:
fs60_home = '/usr/local/freesurfer60'
#FS_tree = 'fs60_subjects'
FS_tree = 'subjects_fs600'

```
%%bash -s  '/usr/local/freesurfer60' '/home/arvid/prj/BrainGut/GE_Signa3T/subjects_fs600'

echo $1
echo $2

FREESURFER_HOME=${1}; export FREESURFER_HOME
PATH=${FREESURFER_HOME}/bin:${PATH}; export PATH
SUBJECTS_DIR=${2}; export SUBJECTS_DIR
FSLDIR=/usr/local/fsl; export FSLDIR
PATH=${FSLDIR}/bin:${PATH}; export PATH
. ${FSLDIR}/etc/fslconf/fsl.sh
source ${FREESURFER_HOME}/SetUpFreeSurfer.sh
```

## How to Convert from FreeSurfer Space Back to Native Anatomical Space
See: https://surfer.nmr.mgh.harvard.edu/fswiki/FsAnat-to-NativeAnat

Question: I have successfully run a subject's data through [FreeSurfer](https://surfer.nmr.mgh.harvard.edu/fswiki/FreeSurfer). 
FreeSurfer creates volumes in 1 mm$^3$, 256$^3$ space, but I want the FreeSurfer results in the space of my original anatomical. How do I do this?<br>

The exact command you use depends on what you want to convert, an image (like brain.mgz) or a segmentation (like aseg.mgz).

For an image:

In [3]:
# cd $SUBJECTS_DIR/<subjid>/mri
# mri_vol2vol --mov brain.mgz --targ rawavg.mgz --regheader --o brain-in-rawavg.mgz --no-save-reg

For a segmentation (aseg.mgz, aparc+aseg.mgz, wmparc.mgz, etc):

In [4]:
# cd $SUBJECTS_DIR/<subjid>/mri
# mri_label2vol --seg aseg.mgz --temp rawavg.mgz --o aseg-in-rawavg.mgz --regheader aseg.mgz

Map the surface to the native space:

In [5]:
# mri_surf2surf --sval-xyz pial --reg register.native.dat rawavg.mgz --tval lh.pial.native --tval-xyz rawavg.mgz --hemi lh --s subjectname

The output will be stored in $SUBJECTS_DIR/subjectname/surf/lh.pial.native and can be viewed with freeview rawavg.mgz -f ../surf/lh.pial.native<br>
To verify that this worked, run

In [6]:
# freeview -v rawavg.mgz -f lh.pial.native

In [7]:
MRI_VOL2VOL = '%s/bin/mri_vol2vol' % (fs60_home)
print(os.popen(MRI_VOL2VOL).read())


mri_vol2vol

  --mov  movvol       : input (or output template with --inv)
  --targ targvol      : output template (or input with --inv)
  --o    outvol       : output volume
  --disp dispvol      : displacement volume

  --reg  register.dat : tkRAS-to-tkRAS matrix   (tkregister2 format)
  --lta  register.lta : Linear Transform Array (usually only 1 transform)
  --lta-inv  register.lta : LTA, invert (may not be the same as --lta --inv with --fstal)
  --fsl  register.fsl : fslRAS-to-fslRAS matrix (FSL format)
  --xfm  register.xfm : ScannerRAS-to-ScannerRAS matrix (MNI format)
  --regheader         : ScannerRAS-to-ScannerRAS matrix = identity
  --mni152reg         : target MNI152 space (need FSL installed)
  --s subject         : set matrix = identity and use subject for any templates

  --inv               : sample from targ to mov

  --tal               : map to a sub FOV of MNI305 (with --reg only)
  --talres resolution : set voxel size 1mm or 2mm (def is 1)
  --talxfm xfmfile    : 

In [8]:
def my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype):
    """
    Ex. 
    cd $SUBJECTS_DIR/<subjid>/mri
    mri_vol2vol --mov brain.mgz --targ rawavg.mgz --regheader --o brain-in-rawavg.mgz --no-save-reg
    --interp interptype : interpolation cubic, trilin, nearest (def is trilin)
    """
    
    #os.chdir(join('%s' % (subj_dir), 'sub_%d_tp%d/mri' % (sub, ses)))
    # fs_mri = join('%s' % (subj_dir), 'sub_%d_tp%d/mri' % (sub, ses))  # if FS_tree = 'fs60_subjects
    fs_mri = join('%s' % (subj_dir), 'subj_%d_tp%d/mri' % (sub, ses))   # if FS_tree = 'subjects_fs600'
    cmd = [
            MRI_VOL2VOL,
            '--mov', '%s/%s.mgz' % (fs_mri, inp_image),
            '--targ', '%s' % (targ_image),
            '--regheader', 
            '--interp', '%s' % (interptype),
            '--o', '%s/sub_%d_tp%d_%s_in_%s.nii.gz' % (out_dir, sub, ses, inp_image, targ_name),
            '--no-save-reg']
          #  ' 2>', error_output_log,'>', output_log]   
    cmd_str = " ".join(cmd)
    #print('cmd_str = \n%s\n' % cmd_str)
        
    # EXECUTE
    os.system(cmd_str)

#### Testing the native space conversion on one subject using the `_T1_biascorr_brain.nii.gz` image obtained from `03-fsl-anat.ipynb` as target image.

In [9]:
working_dir = join(home, 'prj','BrainGut','GE_Signa3T')
subj_dir = '%s/%s' % (working_dir, FS_tree) 
out_dir = '%s/dipy_analysis' % (working_dir)

sub = 101
ses = 1

targ_name = 'native_space'
targ_image = join(out_dir,'sub_%d_tp%d_T1_biascorr_brain.nii.gz' % (sub, ses))

**Use the `my_mri_vol2vol()`function on different source images and masks using approriate interpolation ('trilinear' and 'nearest neighbour')**

In [10]:
interptype = 'trilin'
inp_image = 'brain'
my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)

interptype = 'nearest'
inp_image = 'ribbon'
my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)

interptype = 'nearest'
inp_image = 'aseg'
my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)

interptype = 'nearest'
inp_image = 'wmparc'
my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)

interptype = 'nearest'
inp_image = 'aparc+aseg'
my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)

#### Run the native space conversion on all subjects and sessions using the `_T1_biascorr_brain.nii.gz` image obtained from `03-fsl-anat.ipynb` as target image.

In [12]:
%%time
working_dir = join(home, 'prj','BrainGut','GE_Signa3T')
bids_dir = '%s/bids_toro' % (working_dir)
fsl_anat_dir = '%s/fsl_anat_toro' % (working_dir)
subj_dir = '%s/%s' % (working_dir, FS_tree) 
out_dir = '%s/dipy_analysis' % (working_dir)
targ_name = 'native_space'

nsubj = 30

for i in utils.progressbar(range(nsubj), "Computing: ", 40):
    time.sleep(0.1)
    sub = i+101
    for ses in [1, 2]:
        
        targ_image = join(bids_dir,'sub-%d/ses-%d/anat/sub-%d_ses-%d_T1w.nii.gz' % (sub, ses, sub, ses))
        out_targ_image = join(out_dir,'sub_%d_tp%d_T1w.nii.gz' % (sub, ses))
        
        
        inp_fsl_t1_brain_image = join(fsl_anat_dir,'sub-%d_ses-%d.anat/T1_biascorr_brain.nii.gz' % (sub, ses))
        inp_fsl_t1_brain_mask = join(fsl_anat_dir,'sub-%d_ses-%d.anat/T1_biascorr_brain_mask.nii.gz' % (sub, ses))
        inp_fsl_t1_subcort_seg = join(fsl_anat_dir,'sub-%d_ses-%d.anat/T1_subcort_seg.nii.gz' % (sub, ses))
        
        out_fsl_t1_brain_image = join(out_dir,'sub_%d_tp%d_T1_biascorr_brain.nii.gz' % (sub, ses))
        out_fsl_t1_brain_mask = join(out_dir,'sub_%d_tp%d_T1_biascorr_brain_mask.nii.gz' % (sub, ses))
        out_fsl_t1_subcort_seg = join(out_dir,'sub_%d_tp%d_T1_subcort_seg.nii.gz' % (sub, ses))
        
        shutil.copy2(targ_image, out_targ_image)
        
        shutil.copy2(inp_fsl_t1_brain_image, out_fsl_t1_brain_image)
        shutil.copy2(inp_fsl_t1_brain_mask, out_fsl_t1_brain_mask)
        shutil.copy2(inp_fsl_t1_subcort_seg, out_fsl_t1_subcort_seg)
        
        
        inp_image = 'orig'
        interptype = 'trilin'
        my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)
        
        inp_image = 'brain'
        interptype = 'trilin'
        my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)

        
        inp_image = 'brainmask'
        interptype = 'nearest'
        my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)
                       
        inp_image = 'ribbon'
        interptype = 'nearest'
        my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)

        inp_image = 'aseg'
        interptype = 'nearest'
        my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)

        inp_image = 'wmparc'
        interptype = 'nearest'
        my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)

        inp_image = 'aparc+aseg'
        interptype = 'nearest'
        my_mri_vol2vol(subj_dir, sub, ses, inp_image, targ_image, targ_name, out_dir, interptype)

Computing: [########################################] 30/30
CPU times: user 198 ms, sys: 1.27 s, total: 1.47 s
Wall time: 9min 27s
